In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset= pd.read_csv(r"D:\HopeAI\MLC-Week4\1_RandomForestClassification\Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
indep.shape

(400, 3)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [13]:
#https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
from sklearn.tree import DecisionTreeClassifier       

In [14]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
             'max_features':['auto','sqrt','log2'],
             'splitter':['best','random']}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3, n_jobs=-1, scoring = 'f1_weighted')

grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\mohan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
16 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mohan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mohan\anaconda3\Lib\site-packages\sklearn\base.py", line 1358, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\mohan\anaconda3\Lib\site-packages\sklearn\base.py", line 471, in _validate_params
    

,estimator,DecisionTreeClassifier()
,param_grid,"{'criterion': ['gini', 'entropy'], 'max_features': ['auto', 'sqrt', ...], 'splitter': ['best', 'random']}"
,scoring,'f1_weighted'
,n_jobs,-1
,refit,True
,cv,None
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'entropy'


In [15]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
        

In [16]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
from sklearn.metrics import f1_score
f1_macro = f1_score(y_test, grid_predictions, average = 'weighted')
print("The f1_macro value for the best parameter {}:".format(grid.best_params_),f1_macro)
        

The f1_macro value for the best parameter {'criterion': 'entropy', 'max_features': 'log2', 'splitter': 'best'}: 0.8950415075783872


In [17]:
print("The Confusion Matrix:\n",cm)


The Confusion Matrix:
 [[79  6]
 [ 8 41]]


In [18]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.91      0.93      0.92        85
           1       0.87      0.84      0.85        49

    accuracy                           0.90       134
   macro avg       0.89      0.88      0.89       134
weighted avg       0.89      0.90      0.90       134



In [19]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, grid.predict_proba(x_test)[:,1])


0.8830732292917168

In [20]:
table = pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003767,0.001331,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.005700,0.002377,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.011203,0.004894,0.015849,0.005788,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.826263,0.825959,0.775815,0.906166,0.906166,0.848074,0.050864,5
3,0.008455,0.003590,0.023694,0.005798,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.826263,0.846714,0.793565,0.905069,0.883278,0.850978,0.039735,3
4,0.010727,0.002327,0.019364,0.002002,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.849794,0.907177,0.778067,0.925272,0.862442,0.864550,0.051392,2
5,0.012563,0.003562,0.029706,0.002872,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.849794,0.849057,0.813179,0.865054,0.773585,0.830134,0.033013,7
6,0.002991,0.002821,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.004418,0.003881,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.015311,0.005134,0.035215,0.005511,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.782557,0.870047,0.795256,0.847020,0.923510,0.843678,0.051303,6
9,0.010791,0.004480,0.028098,0.006817,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.864871,0.868752,0.759244,0.886792,0.865054,0.848943,0.045572,4


In [24]:
age_input = float(input("Age:"))
salary_input = float(input("Salary:"))
sex_male_input = int(input("Sex Male 0 or 1:"))
Future_Prediction = grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction={}", format(Future_Prediction))

Age: 20
Salary: 100.00
Sex Male 0 or 1: 0


Future_Prediction={} [1]
